<a href="https://colab.research.google.com/github/harrysharma/Ted_talk_view_prediction/blob/main/Hari_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install contractions

In [ ]:
import numpy as np
import pandas as pd

import nltk
import spacy
import re
from bs4 import BeautifulSoup
import requests
import unicodedata
import contractions

nlp = spacy.load('en_core_web_sm')


In [ ]:
filepath='/content/drive/MyDrive/AlmaBetter/Capstone3_ML_NLP_Regression/cleaned_data.csv'
clean_data=pd.read_csv(filepath)

In [ ]:
df= clean_data[['published_date','daily_views','description']]

In [ ]:
X=df['description']

In [ ]:
df.isna().sum()

published_date    0
daily_views       0
description       0
dtype: int64

In [ ]:
df['published_date'].min()

'2006-06-27'

In [ ]:
train_df = df[df['published_date']<'2019-01-31']
test_df = df[df['published_date']>'2019-01-31']

In [ ]:
train_df.shape, test_df.shape

((3139, 3), (862, 3))

In [ ]:
def remove_special_characters(text, remove_digits= True):
    #text = text.replace('$', 'currency')
    pattern = r'[^\w]+' if not remove_digits else r'[^a-zA-Z]'
    text = re.sub(pattern," ",text)
    text = re.sub(r'\s+',' ', text)
    return text

def remove_accented_characters(text):
    text =  unicodedata.normalize('NFKD',text).encode('ascii','ignore').decode('utf-8','ignore')
    return text

def spacy_lemma(text):
    text = nlp(text)
    new_text = []
    words = [word.lemma_ for word in text]
    for small in words:
        if small == '-PRON-':
            pass
        else:
            new_text.append(small)

    return ' '.join(new_text)

def contractions_text(text):
    return contractions.fix(text)

def stop_words_removal(text, is_lower_case = False, stopwords = None):
    if stopwords == None:
        stopwords = nlp.Defaults.stop_words
    
    if not is_lower_case:
        text = text.lower()
    tokens = nltk.word_tokenize(text)
    new_token = []
    for i in tokens:
        if len(i)<=1:
            pass
        else:
            new_token.append(i)
    
    removed_text = [word for word in new_token if word not in stopwords]
    
    return ' '.join(removed_text)

def join_news(text):
    full_text = []
    for ind in range(len(text)):
        combine_text = []
        for col in range(2,len(text.columns[2:])+2):
            combine_text.append(text.iloc[ind,col])
        full_text.append(' '.join(combine_text))
    return full_text

In [ ]:
train_data = join_news(train_df)
test_data = join_news(test_df)

In [ ]:
train_data[0]

'With the same humor and humanity he exuded in "An Inconvenient Truth," Al Gore spells out 15 ways that individuals can address climate change immediately, from buying a hybrid to inventing a new, hotter brand name for global warming.'

In [ ]:
test_data[0]

'How did the universe begin -- and how is it expanding? CERN physicist Tom Whyntie shows how cosmologists and particle physicists explore these questions by replicating the heat energy and activity of the first few seconds of our universe from right after the Big Bang. [Directed by Dan Abdo and Jason Patterson, narrated by R.J. Glass].'

In [ ]:
import tqdm
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def preprocessor_engine(text):
    corpus =[]
    for sent in tqdm.tqdm(text):
        sent = remove_accented_characters(sent)
        sent = contractions_text(sent)
        sent = remove_special_characters(sent)
        sent = spacy_lemma(sent)
        sent = stop_words_removal(sent)
        corpus.append(sent)
    return corpus

In [ ]:
train_data_pro = preprocessor_engine(train_data)

100%|██████████| 3139/3139 [00:56<00:00, 55.83it/s]


In [ ]:
test_data_pro = preprocessor_engine(test_data)


100%|██████████| 862/862 [00:16<00:00, 51.73it/s]


In [ ]:
train_data_pro[0]

'humor humanity exude inconvenient truth al gore spell way individual address climate change immediately buy hybrid invent new hot brand global warming'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Tfidf_vectorizer=TfidfVectorizer('english',lowercase=True,ngram_range=(1,3),min_df=100,max_features=1000)                 #,min_df=100,max_features=1000

In [ ]:
X_train_tfidf=Tfidf_vectorizer.fit_transform(train_data_pro).toarray()
# features= Tfidf_vectorizer.get_feature_names()
X_test_tfidf=Tfidf_vectorizer.transform(test_data_pro).toarray()

In [ ]:
X_train_tfidf.shape

(3139, 92)

In [ ]:
X_test_tfidf.shape

(862, 92)

In [ ]:
X_train_tfidf = X_train_tfidf[:, :, None]
X_test_tfidf = X_test_tfidf[:, :, None]

In [ ]:
X_train_tfidf.shape

(3139, 92, 1)

In [ ]:
import tensorflow as tf

In [ ]:
# tokenzer = tf.keras.preprocessing.text.Tokenizer(oov_token = '<UNK>')
# tokenzer.fit_on_texts(train_data_pro)

In [ ]:
# #Converting into number token
# train_sequences = tokenzer.texts_to_sequences(train_data_pro)
# test_sequences = tokenzer.texts_to_sequences(test_data_pro)

In [ ]:
# train_sequences

In [ ]:
# print("Vocabulary size ={}".format(len(tokenzer.word_index)))
# print("Number of Documents={}".format(tokenzer.document_count))

In [ ]:
# pd.Series(train_data_pro).apply(lambda x : len(x.split())).max()


In [ ]:
# MAX_SEQUENCE_LENGTH = 73

# train_pad_sequences = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen = MAX_SEQUENCE_LENGTH, padding='post')
# test_pad_sequneces = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen = MAX_SEQUENCE_LENGTH, padding='post')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, RNN, LSTM, GRU, Bidirectional, Embedding, Dropout
from sklearn.metrics import r2_score,mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
y_train = np.array(train_df['daily_views'])
y_test = np.array(test_df['daily_views'])

In [ ]:
y_train

array([ 696.735614  , 2867.64583745,  379.83626656, ..., 1636.09803922,
       4077.25982533, 3002.91466083])

In [ ]:
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [ ]:
y_train

array([[ 696.735614  ],
       [2867.64583745],
       [ 379.83626656],
       ...,
       [1636.09803922],
       [4077.25982533],
       [3002.91466083]])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,r2_score
scaler = MinMaxScaler(feature_range=(0, 1))
y_train = scaler.fit_transform(y_train)
y_test = scaler.transform(y_test)

In [ ]:
def metrics(y_true, y_pred):
    print('r2 Score:', round(r2_score(y_true, y_pred),2))

In [ ]:
# def deep_model(layer_name, epochs=50):
layer_name=LSTM
epochs=50
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# EMBEDDING_DIM = 300 #Dimension for dense embedding for each token
# VOCAB_SIZE = len(tokenzer.word_index)
# model = Sequential()
# model.add((Embedding(input_dim =VOCAB_SIZE+1,output_dim = EMBEDDING_DIM,input_length = MAX_SEQUENCE_LENGTH)))
# model.add((layer_name(256)))
# # model.add((Dense(256,activation = 'relu')))
# model.add(Dense(1,activation = 'sigmoid'))

model = Sequential()
model.add(LSTM(units=3, input_shape = (3139,92, 1), return_sequences = True,output_dim=))
model.add(LSTM(units=3, return_sequences=True))
# model.add(LSTM(units=6, return_sequences=True))
# model.add(LSTM(units=1, return_sequences=True, name='output'))
model.add(Dense(1,activation = 'sigmoid'))




model.compile(optimizer='adam',loss='mse',metrics=[tf.keras.metrics.RootMeanSquaredError()])  
model.summary()
# fit_the_model(model, epochs=50)
early_stop = EarlyStopping(monitor='val_loss', patience=5)
model.fit(X_train_tfidf, y_train, epochs=epochs, validation_data=(X_test_tfidf, y_test), callbacks=[early_stop],verbose=2)
# predictions(model)
train_pred = model.predict(X_train_tfidf,batch_size=92)
test_pred = model.predict(X_test_tfidf)
# metrics(y_train, train_pred)
print('r2 Score:', round(r2_score(y_train, train_pred),2))
print('r2 Score:', round(r2_score(y_test, test_pred),2))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 92, 3)             60        
_________________________________________________________________
lstm_5 (LSTM)                (None, 92, 3)             84        
_________________________________________________________________
dense_2 (Dense)              (None, 92, 1)             4         
Total params: 148
Trainable params: 148
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
99/99 - 10s - loss: 0.1627 - root_mean_squared_error: 0.4034 - val_loss: 0.1100 - val_root_mean_squared_error: 0.3317
Epoch 2/50
99/99 - 6s - loss: 0.0489 - root_mean_squared_error: 0.2212 - val_loss: 0.1009 - val_root_mean_squared_error: 0.3177
Epoch 3/50
99/99 - 6s - loss: 0.0226 - root_mean_squared_error: 0.1502 - val_loss: 0.1049 - val_root_mean_squared_error: 0.3239
E

ValueError: ignored

In [ ]:
# def predictions(model):
#     train_pred = model.predict(train_pad_sequences)
#     test_pred = model.predict(test_pad_sequneces)
#     metrics(y_train, train_pred)
#     # metrics(y_test, test_pred)

# def fit_the_model(model, epochs=50):
#     early_stop = EarlyStopping(monitor='val_loss', patience=5)
#     model.fit(train_pad_sequences, y_train, epochs=epochs, validation_data=(test_pad_sequneces, y_test), callbacks=[early_stop],verbose=0)
#     predictions(model)

In [ ]:
# deep_model(LSTM)

In [ ]:
y_train.shape

(3139, 1)

In [ ]:
train_pred.shape

(3139, 92, 1)

In [ ]:
X_train_tfidf.shape

(3139, 92, 1)

In [ ]:
test_pred.shape

(862, 92, 1)

In [ ]:
train_pred[0].shape

(92, 1)